# талоны, купоны...

In [ ]:
import pandas as pd
import numpy as np
import os

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")

%load_ext lab_black

- rate - количество ставок
- created - создано слотов всего
- occupied - занято слотов всего
- cteated_free - создано слотов общих
- occupied_free - занято слотов общих
- cteated_i - создано слотов, доступных через интернет
- occupied_i - занято слотов, доступных через интернет
- absence - количество неявок
- home - количество посещений на дому
- SPR - создано слотов на одну ставку
- R_EPGU - доля доступных по ЕПГУ от общего количества
- R_b - доля занятых от общего количества
- R_b_EPGU - доля занятых от доступных  по ЕПГУ
- R_ns - доля неявок от занятых

In [62]:
# функции для расчёта показателей


def SPR(row):
    """создано слотов на одну ставку"""
    if row["rate"] == 0:
        return "no data"
    else:
        return round(row["created"] / row["rate"], 3)


def R_EPGU(row):
    """доля доступных по ЕПГУ от общего количества"""
    if row["created"] == 0:
        return "no data"
    else:
        return round((row["created_free"] + row["created_i"]) / row["created"], 3)


def R_b(row):
    """доля занятых от общего количества"""
    if row["created"] == 0:
        return "no data"
    else:
        return round(row["occupied"] / row["created"], 3)


def R_b_EPGU(row):
    """доля занятых от доступных по ЕПГУ"""
    if row["created_free"] + row["created_i"] == 0:
        return "no data"
    else:
        return round(
            (row["occupied_free"] + row["occupied_i"])
            / (row["created_free"] + row["created_i"]),
            3,
        )


def R_ns(row):
    """доля неявок от занятых"""
    if row["occupied"] == 0:
        return "no data"
    else:
        return round(row["absence"] / row["occupied"], 3)

### Формирование обобщенной таблицы

In [39]:
file_name = "Рязань.xlsm"
# sheet = 1
parth = "/home/sk27/work/med/new/coupon/230410/" + file_name
sheets_list = pd.ExcelFile(parth).sheet_names

In [47]:
sheets_list
for i, value in enumerate(sheets_list):
    if len(value.split()) < 3 or value.split()[0] != "с":
        del sheets_list[i]
len(sheets_list)

2

In [4]:
import pandas as pd

In [14]:
parth = "C:/work/test/test.xlsx"

In [15]:
df_test = pd.read_excel(
    parth,
    engine="openpyxl",
    # sheet_name=sheet,
)

In [16]:
df_test.head(10)

,Оперативный отчёт по количеству занятых бирок и фактических посещений,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Территория: Все,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Территория проживания пациента: Все,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Период: с 23.05.2023 по 05.06.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,МО/Отделение,Врач,Должность,Ставка,Количество выложенных к врачу бирок,NaN,в т.ч. обычных,NaN,в т.ч. резервных,NaN,...,NaN,в т.ч платных,NaN,в т. ч. прочие,NaN,Количество посещений в поликлинике,NaN,NaN,Неявка,Количес-\nтво посещ-\nений\nна дому
7,NaN,NaN,NaN,NaN,всего,из них занятых,всего,из них занятых,всего,из них занятых,...,из них занятых,всего,из них занятых,всего,из них занятых,Всего,В т.ч. записаны,Принято без записи,NaN,NaN
8,1,2,3,4,5,6,7,8,9,10,...,20,21,22,23,24,25,26,27,28,29
9,"ГАМУ СО ""СП г. Первоуральск""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_test = df_test.dropna().reset_index(drop=True).drop(index=0)
df_test = df_test.rename(
    columns={
        df_test.columns[0]: "clinic",
        "Unnamed: 1": "division",
        "Unnamed: 2": "name",
        "Unnamed: 3": "doctor_profile",
        "Unnamed: 4": "rate",
        "Unnamed: 5": "created",
        "Unnamed: 6": "occupied",
        "Unnamed: 7": "created_free",
        "Unnamed: 8": "occupied_free",
        "Unnamed: 17": "created_i",
        "Unnamed: 18": "occupied_i",
        "Unnamed: 28": "absence",
        "Unnamed: 29": "home",
    }
)

In [12]:
df_test

,clinic,division,name,doctor_profile,rate,created,occupied,created_free,occupied_free,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,absence


In [51]:
%%time

# просматриваем все файлы в директории "/home/sk27/work/med/new/coupon/230410/"

# читаем названия файлов в директории
path = "/home/sk27/work/med/new/coupon/230410/"
dir_list = os.listdir(path)

# удаляем из списка временные файлы, начинающиеся с "." и файлы, имеющие расширение, отличное от xlsm
for i, value in enumerate(dir_list):
    if value.split(".")[0] == "":
        del dir_list[i]
    if value.split(".")[-1] != "xlsm":
        del dir_list[i]
        
#  пустой файл для хранени общего результата
df_result_full = pd.DataFrame()
for file_name in dir_list:
    parth = "/home/sk27/work/med/new/coupon/230410/" + file_name
    region = file_name.split(".")[0]
    df_result = pd.DataFrame()
    # удаляем листы, названия которых не соответствуют установленным требованиям
    # требования же эти такие: начинается с кириллического "с" и состоит более чем из 3 слов
    sheets_list = pd.ExcelFile(parth).sheet_names
    for i, value in enumerate(sheets_list):
        if len(value.split()) < 3 or value.split()[0] != "с":
            del sheets_list[i]
    for sheet in sheets_list:
        df_coupon = pd.read_excel(
            parth,
            engine="openpyxl",
            sheet_name=sheet,
        )
        # очистка данных
        df_coupon = df_coupon.dropna().reset_index(drop=True).drop(index=0)
        # переименование столбцов
        df_coupon = df_coupon.rename(
            columns={
                df_coupon.columns[0]: "clinic",
                "Unnamed: 1": "division",
                "Unnamed: 2": "name",
                "Unnamed: 3": "doctor_profile",
                "Unnamed: 4": "rate",
                "Unnamed: 5": "created",
                "Unnamed: 6": "occupied",
                "Unnamed: 7": "created_free",
                "Unnamed: 8": "occupied_free",
                "Unnamed: 17": "created_i",
                "Unnamed: 18": "occupied_i",
                "Unnamed: 28": "absence",
                "Unnamed: 29": "home",
            }
        )
        df_coupon["period"] = sheet
        df_result = pd.concat([df_result, df_coupon]).reset_index(drop=True)
    df_result = df_result[["period", 'clinic', 'division', 'name', 'doctor_profile', 'rate', 'created',
           'occupied', 'created_free', 'occupied_free', 'created_i', 'occupied_i', "absence", 'home']]
    df_result.insert(0, "region", region)
    print(f"Обработан {region}, всего {df_result.shape[0]} строк(а, и).")
    df_result_full = pd.concat([df_result_full, df_result]).reset_index(drop=True)
df_result_full = df_result_full.apply(pd.to_numeric, errors="ignore")
df_result_full.to_csv("/home/sk27/work/med/new/coupon/data_result.csv", encoding="utf-8-sig")
print("Данные записаны на диск")

Обработан Иркутск, всего 8191 строк(а, и).
Обработан Марий Эл, всего 3126 строк(а, и).
Обработан Адыгея, всего 1670 строк(а, и).
Обработан ЯНАО, всего 3710 строк(а, и).
Обработан Рязань, всего 3579 строк(а, и).
Обработан ЕАО, всего 274 строк(а, и).
Обработан Коми, всего 4788 строк(а, и).
Данные записаны на диск
CPU times: user 18 s, sys: 47.8 ms, total: 18 s
Wall time: 18.1 s


In [49]:
df_result_full.head(2)

,region,period,clinic,division,name,doctor_profile,rate,created,occupied,created_free,occupied_free,created_i,occupied_i,absence,home
0,Иркутск,с 27.03.2023 по 09.04.2023,"""МЕДСАНЧАСТЬ № 36""",155893. ФЕЛЬДШЕРСКИЕ КАБИНЕТЫ,Бучинская Анастасия Владиславовна,фельдшер,1.0,165,124,32,32,5,5,123,0
1,Иркутск,с 27.03.2023 по 09.04.2023,"""МЕДСАНЧАСТЬ № 36""",155893. ФЕЛЬДШЕРСКИЕ КАБИНЕТЫ,Гуменюк Ирина Александровна,фельдшер,1.0,225,0,0,0,0,0,0,0


### Группировка и расчёт показателей

#### а) без периода

In [74]:
df_result_group = (
    df_result_full.groupby(["region", "clinic", "doctor_profile"]).sum().reset_index()
)

In [76]:
df_result_group["SPR"] = df_result_group.apply(SPR, axis=1)
df_result_group["R_EPGU"] = df_result_group.apply(R_EPGU, axis=1)
df_result_group["R_b"] = df_result_group.apply(R_b, axis=1)
df_result_group["R_b_EPGU"] = df_result_group.apply(R_b_EPGU, axis=1)
df_result_group["R_ns"] = df_result_group.apply(R_b_EPGU, axis=1)

In [78]:
# запсиь на диск
df_result_group.to_csv(
    "/home/sk27/work/med/new/coupon/data_group.csv", encoding="utf-8-sig"
)

#### б) с периодом

In [79]:
df_result_group_period = (
    df_result_full.groupby(["region", "clinic", "period", "doctor_profile"])
    .sum()
    .reset_index()
)

In [80]:
df_result_group_period["SPR"] = df_result_group.apply(SPR, axis=1)
df_result_group_period["R_EPGU"] = df_result_group.apply(R_EPGU, axis=1)
df_result_group_period["R_b"] = df_result_group.apply(R_b, axis=1)
df_result_group_period["R_b_EPGU"] = df_result_group.apply(R_b_EPGU, axis=1)
df_result_group_period["R_ns"] = df_result_group.apply(R_b_EPGU, axis=1)

In [83]:
# запсиь на диск
df_result_group_period.to_csv(
    "/home/sk27/work/med/new/coupon/data_group_period.csv", encoding="utf-8-sig"
)